In [26]:
import ipywidgets as widgets
from IPython.display import display

# Create a text input widget
text_input = widgets.Text(
    value='Hello, World!',
    placeholder='Type something',
    description='Input:',
    disabled=False
)
display(text_input)

Text(value='Hello, World!', description='Input:', placeholder='Type something')

In [ ]:
# ----------------------------
# Audio Recorder Notebook
# ----------------------------

import numpy as np
import wave
import ipyaudioworklet as ipyaudio
from ipywidgets import Output, VBox
from IPython.display import display
import matplotlib.pyplot as plt

# ----------------------------
# Create Recorder
# ----------------------------
rec = ipyaudio.AudioRecorder()
display(rec)

# ----------------------------
# Status Output
# ----------------------------
status_out = Output(layout={'border': '1px solid black', 'padding': '5px'})
status_out.append_stdout("Recorder ready.\n")
display(status_out)

@status_out.capture(clear_output=True)
def status_changed(change):
    print("Status:", change.new)

rec.observe(status_changed, "status")

# ----------------------------
# Function to Save WAV
# ----------------------------
def save_wav_from_recorder(recorder, filename="recording.wav"):
    """Save the recorded audio from recorder into a WAV file."""
    data = recorder.audiodata
    sr = recorder.sampleRate
    
    if data is None or len(data) == 0:
        print("No audio data to save!")
        return None

    # Convert float32 [-1,1] → int16
    data_int16 = (data * 32767).astype(np.int16)

    with wave.open(filename, "wb") as f:
        f.setnchannels(1)
        f.setsampwidth(2)  # int16
        f.setframerate(sr)
        f.writeframes(data_int16.tobytes())

    print(f"Saved: {filename}")
    return filename

# ----------------------------
# Automatic Save and Plot
# ----------------------------
@status_out.capture()
def on_status_change(change):
    if change.new in ("STOPPED", "RECORDED"):
        print("\nRecording finished, saving file...")
        fname = save_wav_from_recorder(rec)
        if fname:
            print("Done.\n")
            
            # Plot waveform
            plt.figure(figsize=(10,3))
            plt.plot(rec.audiodata)
            plt.title("Recorded Audio Waveform")
            plt.xlabel("Samples")
            plt.ylabel("Amplitude")
            plt.show()

rec.observe(on_status_change, "status")


AudioRecorder()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…